In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# cargar los datos
diabetes_dataset = pd.read_csv('diabetes.csv')
data = diabetes_dataset.values

In [ ]:
#Muestro los 5 primeros elementos de dataset
diabetes_dataset.head

In [ ]:
#cantidad de filas y columnas
diabetes_dataset.shape

In [ ]:
#información estadística casos negativos
diabetes_dataset[diabetes_dataset['Outcome']==0].describe()

In [ ]:
#información estadística casos positivos
diabetes_dataset[diabetes_dataset['Outcome']==1].describe()

In [ ]:
#cantidad de diabéticos (1) y no diabéticos (0) en el dataset
diabetes_dataset['Outcome'].value_counts()

In [ ]:
#Crear diagrama de barras
sns.countplot(x='Outcome', data=diabetes_dataset)
plt.title('Cantidad de pacientes diabéticos y no diabéticos')
plt.xlabel('Estado')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
#el % de glucosa en sangre en diabéticos es mayor que los que no lo son.
#A su vez, la personas de mayor edad son más suceptibles a tener diabetes.
diabetes_dataset.groupby('Outcome').mean()

In [ ]:
# gráfico de dispersión que indica la relación entre la edad y el IMC en sangre de los pacientes diabéticos y no diabéticos
sns.scatterplot(x='Age', y='BMI', hue='Outcome', data=diabetes_dataset)
plt.title('Relación entre edad y el IMC')
plt.xlabel('Edad')
plt.ylabel('IMC')
plt.show()


In [ ]:
# diagrama de cajas y bigotes que compara la distribución del nivel de glucosa en sangre entre pacientes diabéticos
#y no diabéticos en el conjunto de datos. 
sns.boxplot(x='Outcome', y='Glucose', data=diabetes_dataset, fliersize=3)
plt.title('Distribución del nivel de glucosa en sangre para pacientes diabéticos y no diabéticos', fontsize=14)
plt.xlabel('Diabético', fontsize=12)
plt.ylabel('Nivel de glucosa', fontsize=12)
plt.figure(figsize=(8,6))
plt.show()

In [ ]:
# gráfico de dispersión que indica la relación entre la edad y el nivel de glucosa en sangre de los pacientes diabéticos y no diabéticos
sns.scatterplot(x='Age', y='Glucose', hue='Outcome', data=diabetes_dataset)
plt.title('Relación entre edad y nivel de glucosa en sangre')
plt.xlabel('Edad')
plt.ylabel('Nivel de glucosa en sangre')
plt.show()

In [ ]:
# gráfico de dispersión que indica la relación entre la insulina y el nivel de glucosa en sangre de los pacientes diabéticos y no diabéticos
sns.scatterplot(x='Glucose', y='Insulin', hue='Outcome', data=diabetes_dataset)
plt.title('Relación entre nivel de glucosa e insulina en sangre')
plt.xlabel('Nivel de glucosa en sangre')
plt.ylabel('Nivel de insulina en sangre')
plt.show()

In [ ]:
# Me quedo con las columnas relevantes y las traduzco al español
diabetes_df_limpio = diabetes_dataset.loc[:, ['Age', 'Glucose', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Outcome']]
diabetes_df_limpio = diabetes_df_limpio.rename(columns={
    'Age': 'Edad',
    'Glucose': 'Glucosa',
    'Insulin': 'Insulina',
    'BMI': 'IMC',
    'DiabetesPedigreeFunction': 'Historial Familiar de Diabetes',
    'Outcome': 'Resultado'
})

diabetes_df_limpio

In [ ]:
#matriz de correlación entre las variables
corr = diabetes_df_limpio.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
sns.pairplot(data=diabetes_df_limpio, hue='Resultado', corner=True)

In [ ]:
#separo los datos y los labels
x = diabetes_df_limpio.drop(columns='Resultado', axis=1)
y = diabetes_df_limpio['Resultado']

In [ ]:
# divido los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# normalizar los datos de entrenamiento y transformo los datos de prueba en la misma escala
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# creo el modelo de regresión logística y lo ajusto a los datos de entrenamiento
model = LogisticRegression()
model.fit(X_train, y_train)

# evalúo el modelo en los datos de prueba
score = model.score(X_test, y_test)

# imprimo la precisión del modelo
print(f'Precisión del modelo: {score:.3f}')

In [ ]:
# Crear los widgets de entrada
glucosa = widgets.FloatText(description='Glucosa (mg/dL):')
insulina = widgets.FloatText(description='Insulina (mu U/mL):')
imc = widgets.FloatText(description='IMC:')
pedigri = widgets.FloatText(description='Pedigrí:')
edad = widgets.FloatText(description='Edad (años):')

# Crear una función que se activa cuando se presiona el botón de "Predecir"
def predecir(sender):
    try:
        # Comprobar que los valores ingresados son válidos
        if edad.value <= 0 or glucosa.value <= 0 or insulina.value < 0 or imc.value <= 0 or pedigri.value < 0:
            print("Error: Asegúrese de ingresar valores válidos")
            return

        input_data = np.array([edad.value, glucosa.value, insulina.value, imc.value, pedigri.value])

        # cambio la forma del arreglo ya que estoy realizando una predicción para una única instancia de datos
        input_data_reshaped = input_data.reshape(1,-1)

        # estandarizar la entrada
        std_data = scaler.transform(input_data_reshaped)

        # hacer una predicción en los datos estandarizados
        prediction = model.predict(std_data)
        proba = model.predict_proba(std_data)

        # imprimir la predicción y la probabilidad estimada
        print('----------------------------------------')
        if prediction[0] == 0:
            print('La persona no tiene diabetes.')
        else:
            print('La persona tiene diabetes.')
        print(f"Probabilidad estimada de no tener diabetes: {proba[0][0]:.3f}")
        print(f"Probabilidad estimada de tener diabetes: {proba[0][1]:.3f}")
        print('----------------------------------------')
        print(f"Edad: {edad.value} años")
        print(f"Nivel de glucosa en sangre: {glucosa.value} mg/dL")
        print(f"Nivel de insulina en sangre: {insulina.value} mu U/mL")
        print(f"Índice de masa corporal (IMC): {imc.value}")
        print(f"Pedigrí de diabetes en la familia: {pedigri.value}")
        print('----------------------------------------')
    except:
        print("Error: Asegúrese de ingresar números en todas las entradas")



# Crear el botón de "Predecir"
boton_predecir = widgets.Button(description='Predecir')
boton_predecir.on_click(predecir)

# Mostrar los widgets y el botón
display(edad, glucosa, insulina, imc, pedigri, boton_predecir)

